# Pune Weather Stations

## Import necessary packages

In [148]:
from iudx.entity.Entity import Entity

import pandas as pd
import numpy as np
import json
from datetime import date, datetime, timedelta

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from scipy.interpolate import griddata
import geojsoncontour

import ipywidgets as widgets
from ipywidgets import Layout

import warnings

In [149]:
aqm_id="datakaveri.org/04a15c9960ffda227e9546f3f46e629e1fe4132b/rs.iudx.org.in/pune-env-aqm"
ws_id="datakaveri.org/04a15c9960ffda227e9546f3f46e629e1fe4132b/rs.iudx.org.in/pune-env-weather"
ndays=3

### Fetch and prepare weather Stations data

In [150]:
entity=Entity(entity_id=ws_id)
latest_measures=entity.latest().reset_index(drop=True)
end_time = latest_measures['observationDateTime'].sort_values(ascending=False).reset_index(drop=True)[0]
start_time = (end_time - timedelta(days=ndays,hours=6))
measures = entity.during_search(
    start_time=start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
    end_time=end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
)
measures['observationDateTime']=measures['observationDateTime'].apply(lambda x:x.tz_localize(None))
latest_measures['observationDateTime']=latest_measures['observationDateTime'].apply(lambda x:x.tz_localize(None))
rs_coordinates={}
rs_label={}
for res in entity.resources:
    rs_coordinates[res['id']]=res['location']['geometry']['coordinates']
    rs_label[res['id']]=res['name']
latest_measures['x_co']=latest_measures['id'].apply(lambda id:rs_coordinates[id][0])
latest_measures['y_co']=latest_measures['id'].apply(lambda id:rs_coordinates[id][1])
measures['x_co']=measures['id'].apply(lambda id:rs_coordinates[id][0])
measures['y_co']=measures['id'].apply(lambda id:rs_coordinates[id][1])
measures['label']=measures['id'].apply(lambda id:rs_label[id])
latest_measures['label']=measures['id'].apply(lambda id:rs_label[id])

In [151]:
measures.isnull().sum()

airQualityIndex                   0
id                                0
uv                              710
aqiMajorPollutant                 0
relativeHumidityForecast        142
deviceName                        0
airQualityIndexForecast           0
airTemperature                  710
relativeHumidity                710
observationDateTime               0
precipitation                   281
airTemperatureForecast          142
aqiMajorPollutantForecast         0
precipitationForecast           142
airTemperature.avgOverTime      349
relativeHumidity.avgOverTime    349
uv.avgOverTime                  568
x_co                              0
y_co                              0
label                             0
dtype: int64

## Visualizations

In [152]:
# plot the measures of a proprty over ndays for the resource with the latest recording
def timeSeriesVis1(column_name, ndays):
    sensor_id = measures.sort_values(by='observationDateTime',ascending=False).reset_index(drop=True)['id'][0]
    single_resource_data = measures.query(f"id == '{sensor_id}'")
    sensor_coordinates=[]
    warnings.filterwarnings('ignore')
    for res in entity.resources:
        if res['id']==sensor_id:
            sensor_coordinates=res['location']['geometry']['coordinates']
    single_resource_data['day']=single_resource_data['observationDateTime'].apply(lambda x:x.strftime('%A'))
    
    fig = px.box(
        single_resource_data, 
        x="day", 
        y=column_name,
        points='all'
    )
    display(widgets.HTML(f'<center style="font-size:14px">Temporal sensor reading for \n {column_name.upper()} from {start_time.date()} to {end_time.date()} for resource at {sensor_coordinates}<center>'))
    fig.update_layout(
        xaxis_title="Observed Timestamp",
        yaxis_title="Sensor reading for "+column_name.upper(),
        font=dict(
            size=12
        )
    )
    fig.show()
timeSeriesVis1('airQualityIndex',ndays)


HTML(value='<center style="font-size:14px">Temporal sensor reading for \n AIRQUALITYINDEX from 2021-03-26 to 2…

In [153]:
# plot the measures of a proprty over ndays for all resources
def timeSeriesVis2(col, ndays):
    column_name=col
    fig = px.line(
        measures, 
        x="observationDateTime", 
        y=column_name,
        color='deviceName'
    )
    display(widgets.HTML(f'<center style="font-size:14px">Temporal sensor reading for {col.upper()} from {start_time.date()} to {end_time.date()} of all sensors<center>'))
    fig.update_layout(
        xaxis_title="Observed Timestamp",
        yaxis_title="Sensor reading for "+col.upper(),
        font=dict(
            size=12
        )
    )
    fig.update_xaxes(rangeslider_visible=True)
    fig.show()
timeSeriesVis2('airTemperature.avgOverTime',ndays)


HTML(value='<center style="font-size:14px">Temporal sensor reading for AIRTEMPERATURE.AVGOVERTIME from 2021-03…

In [154]:
# plot a bar chart for the latest measures of a property at all active resources
def simpleVis1(col):
    column_name=col
    display(widgets.HTML(f'<center style="font-size:14px">Latest temporal sensor reading for {col.upper()} of all sensors<center>'))
    fig = px.bar(latest_measures, x='deviceName', y=column_name)
    fig.update_layout(
        xaxis_title="Sensor Id",
        yaxis_title="Sensor reading for "+col.upper(),
        font=dict(
            size=12
        )
    )
    fig.show()
simpleVis1('airQualityIndex')


HTML(value='<center style="font-size:14px">Latest temporal sensor reading for AIRQUALITYINDEX of all sensors<c…

In [155]:
#spatial bubble chart over Pune
maxval=max(list(filter(None,latest_measures['airQualityIndex'])))
minval=min(list(filter(None,latest_measures['airQualityIndex'])))
geomap2 = folium.Map([latest_measures['y_co'].mean(), latest_measures['x_co'].mean()], zoom_start=12, tiles="cartodbpositron")
for res in entity.resources:
    entity_id = res["id"]
    try:
        val=latest_measures[latest_measures['id']==entity_id]['airQualityIndex'].values[0]
        if val is not None and val>0:
            folium.Circle(
              [res["location"]["geometry"]["coordinates"][1], res["location"]["geometry"]["coordinates"][0]],
              radius=2000*(val-minval)/(maxval-minval),
              popup = f'{res["name"].upper()} Air Quality Index: {str(val)}',
              color='b',
              fill_color=('red' if ((val-minval)/(maxval-minval))>0.6 else 'blue'),
              fill=True,
              fill_opacity=0.4
              ).add_to(geomap2)
    except:
        pass
display(geomap2)

### Fetching Pune's AQM data for comparison

In [156]:
entity2=Entity(entity_id=aqm_id)
latest_measures2=entity2.latest().reset_index(drop=True)
end_time2 = latest_measures2['observationDateTime'].sort_values(ascending=False).reset_index(drop=True)[0]
start_time2 = (end_time2 - timedelta(days=ndays,hours=6))
measures2 = entity2.during_search(
    start_time=start_time2.strftime("%Y-%m-%dT%H:%M:%SZ"),
    end_time=end_time2.strftime("%Y-%m-%dT%H:%M:%SZ"),
)
measures2['observationDateTime']=measures2['observationDateTime'].apply(lambda x:x.tz_localize(None))
latest_measures2['observationDateTime']=latest_measures2['observationDateTime'].apply(lambda x:x.tz_localize(None))
rs_coordinates={}
rs_label={}
for res in entity2.resources:
    rs_coordinates[res['id']]=res['location']['geometry']['coordinates']
    rs_label[res['id']]=res['name']
latest_measures2['x_co']=latest_measures2['id'].apply(lambda id:rs_coordinates[id][0])
latest_measures2['y_co']=latest_measures2['id'].apply(lambda id:rs_coordinates[id][1])
measures2['x_co']=measures2['id'].apply(lambda id:rs_coordinates[id][0])
measures2['y_co']=measures2['id'].apply(lambda id:rs_coordinates[id][1])
measures2['label']=measures2['id'].apply(lambda id:rs_label[id])
latest_measures2['label']=measures2['id'].apply(lambda id:rs_label[id])

In [157]:
from math import cos, asin, sqrt, pi

#function to calculate distance between two coordinates in kilometers
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...

In [158]:
#pairing the a weather station with the nearest AQM sensor
sensor_map={}
for wstation in entity.resources:
    lat1=wstation['location']['geometry']['coordinates'][1]
    lon1=wstation['location']['geometry']['coordinates'][0]
    mindist=10000
    nearest_res=None
    for res in entity2.resources:
        lat=res['location']['geometry']['coordinates'][1]
        lon=res['location']['geometry']['coordinates'][0]
        dist=distance(lat1,lon1,lat,lon)
        if mindist>dist:
            mindist=dist
            lat2=lat
            lon2=lon
            nearest_res=res
    sensor_map[wstation['id']]=nearest_res['id']

### Visualizing with data from closest AQM sensor

In [159]:
for res in entity.resources:
    print(res['name'])

Nigadi
Manjari
Bhumkar_Chowk
Alandi
Hadapsar
Bhosari
Pashan
Shivajinagar
Lohgaon
Katraj


In [160]:
device_name='Pashan'
column='airTemperature.avgOverTime'
column2='airTemperature.avgOverTime'
single_resource_data = measures.query(f"deviceName == '{device_name}'")
lat1=single_resource_data.iloc[0]['y_co']
lon1=single_resource_data.iloc[0]['x_co']
mindist=10000
lat2=0
lon2=0
nearest_res=None
for res in entity2.resources:
    lat=res['location']['geometry']['coordinates'][1]
    lon=res['location']['geometry']['coordinates'][0]
    dist=distance(lat1,lon1,lat,lon)
    if mindist>dist:
        mindist=dist
        lat2=lat
        lon2=lon
        nearest_res=res
print(f'Nearest sensor is at {nearest_res["location"]["address"]}')
print(f'Distance between: {mindist} kms')

single_resource_data2 = measures2.query(f"id == '{nearest_res['id']}'")

geomap3 = folium.Map([lat1,lon1], zoom_start=12, tiles="cartodbpositron")
folium.Marker([lat1,lon1]).add_to(geomap3)
folium.Marker([lat2,lon2]).add_to(geomap3)
display(geomap3)

fig=go.Figure()
#fig1=go.Figure()
fig.add_trace(go.Scatter(x=single_resource_data['observationDateTime'],
                        y=single_resource_data[column],
                        name=f'{column} Weather Station',
                        line=dict(color='firebrick')))
fig.add_trace(go.Scatter(x=single_resource_data2['observationDateTime'],
                        y=single_resource_data2[column2],
                        name=f'{column2} AQM Sensor',
                        line=dict(color='royalblue')))
fig.update_layout(title=f'{column} at weather station and {column2} at AQM sensor',
                 xaxis_title='Timestamp',
                 yaxis_title=f'{column}')
fig.update_xaxes(rangeslider_visible=True)
fig.show()
#fig1.show()

Nearest sensor is at Kothrud PMPML BusDepot_48, Paud Road Pune Maharashtra
Distance between: 2.4917337466942526 kms


### Functions to plot contours for both weather stations and AQM data

In [161]:
def wsSpatialVis1(col):
    column_name=col
    x_orig = []
    y_orig = []
    zs = []
    for res in entity.resources:
        try:
            val = latest_measures[latest_measures["id"] == res["id"]][column_name].values[0]
            if val is not None and val>0:
                zs.append(val)
                x_orig.append(res["location"]["geometry"]["coordinates"][0])
                y_orig.append(res["location"]["geometry"]["coordinates"][1])
        except:
            pass
    x_orig = np.array(x_orig)
    y_orig = np.array(y_orig)
    zs = np.array(zs)
    # Initialize the map
    geomap1 = folium.Map([y_orig.mean(), x_orig.mean()], zoom_start=11, tiles="cartodbpositron")
    for res in entity.resources:
        entity_id = res["id"]
        try:
            val=latest_measures[latest_measures['id']==entity_id][column_name].values[0]
            if val is not None and val>0:
                folium.Marker([res["location"]["geometry"]["coordinates"][1], res["location"]["geometry"]["coordinates"][0]],
                          tooltip=f'{col.upper()}: {str(val)}').add_to(geomap1)
        except:
            pass
    # Make lat and lon linspace
    y_arr = np.linspace(np.min(y_orig), np.max(y_orig), 100)
    x_arr = np.linspace(np.min(x_orig), np.max(x_orig), 100)
    # Make mesh grid
    x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
    # Perform cubic interpolation
    z_mesh = griddata((x_orig, y_orig), zs, (x_mesh, y_mesh), method='cubic')
    # Number of levels of colors
    levels = 20
    contourf=plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.5, 
                              cmap="bwr", linestyles='None', vmin=0, vmax=100)
    plt.close()
    # Convert matplotlib contourf to geojson
    geojson = geojsoncontour.contourf_to_geojson(
        contourf=contourf,
        min_angle_deg=3.0,
        ndigits=5,
        stroke_width=1,
        fill_opacity=0.5)
    # Plot the contour plot on folium
    folium.GeoJson(
        geojson,
        style_function=lambda x: {
            'color':     x['properties']['stroke'],
            'weight':    x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity':   0.6,
        }).add_to(geomap1)
    # Show map
    display(geomap1)
    
def aqmSpatialVis1(col):
    column_name=col
    x_orig = []
    y_orig = []
    zs = []
    for res in entity2.resources:
        if res['id'] in sensor_map.values():
            try:
                val = latest_measures2[latest_measures2["id"] == res["id"]][column_name].values[0]
                if val is not None and int(val)>0:
                    zs.append(val)
                    x_orig.append(res["location"]["geometry"]["coordinates"][0])
                    y_orig.append(res["location"]["geometry"]["coordinates"][1])
            except:
                pass
    x_orig = np.array(x_orig)
    y_orig = np.array(y_orig)
    zs = np.array(zs)
    # Initialize the map
    geomap1 = folium.Map([y_orig.mean(), x_orig.mean()], zoom_start=11, tiles="cartodbpositron")
    for res in entity2.resources:
        entity_id = res["id"]
        if res['id'] in sensor_map.values():
            try:
                val=latest_measures2[latest_measures2['id']==entity_id][column_name].values[0]
                if val is not None and int(val)>0:
                    folium.Marker([res["location"]["geometry"]["coordinates"][1], res["location"]["geometry"]["coordinates"][0]],
                              tooltip=f'{col.upper()}: {str(val)}').add_to(geomap1)
            except:
                pass
    # Make lat and lon linspace
    y_arr = np.linspace(np.min(y_orig), np.max(y_orig), 100)
    x_arr = np.linspace(np.min(x_orig), np.max(x_orig), 100)
    # Make mesh grid
    x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
    # Perform cubic interpolation
    z_mesh = griddata((x_orig, y_orig), zs, (x_mesh, y_mesh), method='cubic')
    # Number of levels of colors
    levels = 20
    contourf=plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.5, 
                              cmap="bwr", linestyles='None', vmin=0, vmax=100)
    plt.close()
    # Convert matplotlib contourf to geojson
    geojson = geojsoncontour.contourf_to_geojson(
        contourf=contourf,
        min_angle_deg=3.0,
        ndigits=5,
        stroke_width=1,
        fill_opacity=0.5)
    # Plot the contour plot on folium
    folium.GeoJson(
        geojson,
        style_function=lambda x: {
            'color':     x['properties']['stroke'],
            'weight':    x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity':   0.6,
        }).add_to(geomap1)
    # Show map
    display(geomap1)

In [162]:
wsSpatialVis1('airQualityIndex')
aqmSpatialVis1('airQualityIndex')

### Calculating correlation between weather station data and AQM sensor data

In [163]:
corr_list=[]
for w_id in sensor_map.keys():
    single_res_data=measures.query(f'id == "{w_id}"')
    single_res_data2=measures2.query(f'id == "{sensor_map[w_id]}"').reset_index(drop=True)

    indexlist=[]
    try:
        for time in single_res_data['observationDateTime']:
            indexlist.append(pd.DataFrame(single_res_data2['observationDateTime']).set_index('observationDateTime').index.get_loc(time, method='nearest'))
        req_single_res_data2=single_res_data2.iloc[indexlist]

        corr_list.append(
            single_res_data['airQualityIndex'].corr(
                req_single_res_data2['airQualityIndex'].astype(float)))
    except:
        corr_list.append(0)
        print(f'{w_id} correlation not found')

datakaveri.org/04a15c9960ffda227e9546f3f46e629e1fe4132b/rs.iudx.org.in/pune-env-weather/Shivajinagar correlation not found


In [164]:
corr_list

[-0.9736676403347939,
 -0.2769861544056646,
 0.21212799975977523,
 1.0,
 -0.9999999999999999,
 -0.26041032586927804,
 -0.749903543180104,
 0,
 -0.3278488376861446,
 0.9473165606597677]

In [169]:
#visualizing the correlation
fig = go.Figure(data=[go.Surface(z=corr_list,
                                 x=[x['location']['geometry']['coordinates'][0] for x in entity.resources],
                                 y=[x['location']['geometry']['coordinates'][1] for x in entity.resources])])
fig.show()